# 데이터 확인

In [1]:
import pymysql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import config 
from config import DB_CONFIG
import pymysql  

In [2]:
# MySQL 연결
conn = pymysql.connect(
    host=DB_CONFIG['host'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database'],
    port=DB_CONFIG['port']
)

car_siren = f"SELECT * FROM merge50_car_siren"
car_horn = f"SELECT * FROM merge50_car_horn"
car_driving = f'SELECT * FROM merge50_car_driving'
motorcycle_horn  = f'SELECT * FROM merge50_motorcycle_horn'
motorcycle_driving  = f'SELECT * FROM merge50_motorcycle_driving'

car_siren_df =  pd.read_sql(car_siren, conn)
car_horn_df = pd.read_sql(car_horn, conn) 
car_driving_df = pd.read_sql(car_driving, conn) 
motorcycle_horn_df= pd.read_sql(motorcycle_horn, conn) 
motorcycle_driving_df = pd.read_sql(motorcycle_driving, conn) 

conn.close()

In [3]:
print(f'car_siren 데이터 개수: {car_siren_df.shape[0]}')
print(f'car_horn 데이터 개수: {car_horn_df.shape[0]}')
print(f'car_driving 데이터 개수: {car_driving_df.shape[0]}')
print(f'motorcycle_horn 데이터 개수: {motorcycle_horn_df.shape[0]}')
print(f'motorcycle_driving 데이터 개수: {motorcycle_driving_df.shape[0]}')

car_siren 데이터 개수: 1990
car_horn 데이터 개수: 3189
car_driving 데이터 개수: 1682
motorcycle_horn 데이터 개수: 4560
motorcycle_driving 데이터 개수: 4735


In [4]:
car_siren_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fileName       1990 non-null   object 
 1   labelName      1990 non-null   object 
 2   area_start     1990 non-null   float64
 3   area_end       1990 non-null   float64
 4   category_01    1990 non-null   object 
 5   category_02    1990 non-null   object 
 6   category_03    1990 non-null   object 
 7   decibel        1990 non-null   int64  
 8   soundQuality   1990 non-null   object 
 9   subCategory    1990 non-null   object 
 10  bitRate        1990 non-null   object 
 11  duration       1990 non-null   float64
 12  fileFormat     1990 non-null   object 
 13  fileSize       1990 non-null   int64  
 14  recodingType   1990 non-null   object 
 15  sampleRate     1990 non-null   object 
 16  acqDevice      1990 non-null   object 
 17  acqMethod      1990 non-null   object 
 18  acqType 

In [5]:
car_horn_df.iloc[ : ,-50:] 

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,mfcc_41,mfcc_42,mfcc_43,mfcc_44,mfcc_45,mfcc_46,mfcc_47,mfcc_48,mfcc_49,mfcc_50
0,-391.00430,107.641975,64.204640,33.718952,20.630340,13.337987,9.100019,8.873220,9.340549,8.287990,...,-1.194056,-0.757743,-2.858810,-5.101422,-0.198150,7.001447,6.203896,-2.862313,-10.336964,-8.593492
1,-233.66284,87.917620,45.434906,19.435001,9.098841,32.400368,5.817845,9.204194,14.611247,20.294502,...,-7.923782,8.347244,-2.440833,-17.408949,-5.891588,22.778133,21.374588,-19.310514,-26.457247,4.794274
2,-318.16153,126.123180,43.729805,-11.429483,-5.223235,13.667998,0.934844,10.497743,2.401143,22.281258,...,-5.953082,1.538178,-0.032047,-10.759938,-10.296665,14.938017,19.099138,-5.256958,-27.792421,-7.377015
3,-385.20926,140.265410,57.263832,-7.932145,-3.877982,14.155136,6.298158,11.397833,2.566783,15.076704,...,-5.018665,2.996135,-0.142866,-8.995171,-4.227169,8.116351,11.163322,-4.992955,-16.359087,-5.441961
4,-382.12450,150.004900,61.439040,-17.890905,-7.740589,15.947204,3.806582,7.909450,2.633213,14.426663,...,-5.330876,2.107300,1.768516,-10.489394,-4.815802,9.592706,12.866017,-3.469832,-20.286922,-6.563427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3184,-99.19382,119.270660,-76.342830,14.679804,-26.403290,5.220899,-15.944427,-0.425459,-14.378425,-2.452169,...,1.322109,-1.603073,-0.969422,-0.030491,-3.283236,-4.058084,-2.597992,-0.028827,1.399276,-4.661576
3185,-292.45557,203.069000,-53.426810,37.105576,6.516268,10.316400,11.667080,1.168588,17.977980,5.434537,...,0.412159,-0.795219,-1.426503,-3.167590,-2.035457,0.571453,4.668703,3.238892,-5.142219,-4.995093
3186,-198.79451,201.686460,-87.792480,-11.408583,-14.657739,-5.088332,-0.388612,-13.570362,6.362044,-11.089597,...,-1.205375,0.633744,2.143693,-4.530044,-7.654323,3.844430,9.282047,2.180258,-6.707889,-3.188458
3187,-247.43971,186.716000,-45.675808,47.491760,18.612213,16.028240,8.016017,3.121070,13.013415,4.386095,...,0.659112,-2.346271,-1.647404,-0.503469,0.055166,-0.353136,-0.114590,1.304396,0.613524,-0.592904


In [7]:
#데이터프레임 병합 
df_combined = pd.concat([car_horn_df, car_siren_df,car_driving_df,motorcycle_horn_df,motorcycle_driving_df], ignore_index=True)

print(f'df_combined 행 개수: {len(df_combined)}') 
df_combined.head()

df_combined 행 개수: 16156


,fileName,labelName,area_start,area_end,category_01,category_02,category_03,decibel,soundQuality,subCategory,...,mfcc_41,mfcc_42,mfcc_43,mfcc_44,mfcc_45,mfcc_46,mfcc_47,mfcc_48,mfcc_49,mfcc_50
0,1.car_horn_87900_1.wav,1.car_horn_87900_1.wav,2.0,58.719,교통소음,자동차,차량경적,116,정상,소형차경적,...,-1.194056,-0.757743,-2.858810,-5.101422,-0.198150,7.001447,6.203896,-2.862313,-10.336964,-8.593492
1,1.car_horn_87964_1.wav,1.car_horn_87964_1.wav,2.0,59.747,교통소음,자동차,차량경적,102,정상,소형차경적,...,-7.923782,8.347244,-2.440833,-17.408949,-5.891588,22.778133,21.374588,-19.310514,-26.457247,4.794274
2,1.car_horn_87987_1.wav,1.car_horn_87987_1.wav,2.0,59.217,교통소음,자동차,차량경적,112,정상,소형차경적,...,-5.953082,1.538178,-0.032047,-10.759938,-10.296665,14.938017,19.099138,-5.256958,-27.792421,-7.377015
3,1.car_horn_88000_1.wav,1.car_horn_88000_1.wav,2.0,57.389,교통소음,자동차,차량경적,112,정상,소형차경적,...,-5.018665,2.996135,-0.142866,-8.995171,-4.227169,8.116351,11.163322,-4.992955,-16.359087,-5.441961
4,1.car_horn_88001_1.wav,1.car_horn_88001_1.wav,2.0,58.207,교통소음,자동차,차량경적,122,정상,소형차경적,...,-5.330876,2.107300,1.768516,-10.489394,-4.815802,9.592706,12.866017,-3.469832,-20.286922,-6.563427


# lightGBM + ODD

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# 레이블(label)과 피처(features) 분리
X = df_combined.iloc[:, -50:]  # 마지막 13개 컬럼을 특성(MFCC)으로 사용
y = df_combined['category_03'].astype('category').cat.codes  # 범주형 데이터를 숫자로 변환

# 범주형 데이터(레이블) 숫자로 변환
y = y.astype('category').cat.codes

# 클래스 레이블 매핑
class_labels = df_combined['category_03'].astype('category').cat.categories

# 데이터셋 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(set(y))
print(num_classes)

# LightGBM 모델 정의 및 학습
model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# 평가 결과 출력
acc = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도: {acc:.4f}")

# 혼동 행렬 및 분류 보고서 출력
print("\nConfusion Matrix:")
conf_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels.tolist()))



5
LightGBM 정확도: 0.9449

Confusion Matrix:
        이륜차경적  이륜차주행음  차량경적  차량사이렌  차량주행음
이륜차경적     902       2    44      1      0
이륜차주행음      0     885     3      1     26
차량경적       24      10   582      1      4
차량사이렌       0      11     3    401      7
차량주행음       0      40     0      1    284

Classification Report:
              precision    recall  f1-score   support

       이륜차경적       0.97      0.95      0.96       949
      이륜차주행음       0.93      0.97      0.95       915
        차량경적       0.92      0.94      0.93       621
       차량사이렌       0.99      0.95      0.97       422
       차량주행음       0.88      0.87      0.88       325

    accuracy                           0.94      3232
   macro avg       0.94      0.94      0.94      3232
weighted avg       0.95      0.94      0.95      3232



In [8]:
import lightgbm as lgb

# 모델 학습
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

# 모델 저장 (Booster 객체 사용)
model.booster_.save_model('lgbm_model.txt')

print("모델이 성공적으로 저장되었습니다!")

모델이 성공적으로 저장되었습니다!


# df_etc 기타 소음 (열차, 비행기) 확인

In [9]:
import os
import json
import librosa
import numpy as np
import pandas as pd

data_dir = "/home/ubuntu/data/etc_noise_data_test"

# 기존 데이터프레임이 존재하는지 확인
try:
    df_combined
except NameError:
    df_combined = pd.DataFrame()  # 없으면 빈 데이터프레임 생성

json_data = []
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

            annotations = data.get("annotations", [])
            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

            # JSON 파일명에서 확장자 제거
            base_name = os.path.splitext(file)[0]

            json_data.append({
                "baseName": base_name,  # 확장자 없는 파일명 저장
                "fileName": data["audio"]["fileName"],
                "labelName": annotation.get("labelName", None),
                "area_start": annotation.get("area", {}).get("start", None),
                "area_end": annotation.get("area", {}).get("end", None),
                "category_01": annotation.get("categories", {}).get("category_01", None),
                "category_02": annotation.get("categories", {}).get("category_02", None),
                "category_03": annotation.get("categories", {}).get("category_03", None),
                "decibel": annotation.get("decibel", None),
                "soundQuality": annotation.get("soundQuality", None),
                "subCategory": annotation.get("subCategory", None),
            })

df_json = pd.DataFrame(json_data)

# WAV 파일에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=50):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

wav_data = []
for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        base_name = "_".join(file.split("_")[:-1])  # "_1.wav" 부분 제거하고 기본 파일명 추출
        mfcc_features = extract_mfcc(file_path)

        wav_data.append({
            "baseName": base_name,  # JSON과 매칭할 기본 파일명 저장
            **{f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfcc_features)}
        })

df_wav = pd.DataFrame(wav_data)

# # JSON 데이터와 WAV 데이터 병합 (baseName 기준)
df_etc = pd.merge(df_json, df_wav, on="baseName", how="left").drop(columns=["baseName"])

# # 기존 데이터프레임 df_combined에 추가
df_combined2 = pd.concat([df_combined, df_etc], ignore_index=True)

# # 결과 확인
# print(df_combined.head())

In [10]:
# 📌 4️⃣ OOD 데이터 로드 & 전처리 (MFCC 특징 사용)
ood_X_test = df_etc.iloc[:, -50:]  # OOD 데이터의 MFCC 13개 컬럼 사용

# 📌 5️⃣ Softmax 기반 OOD 탐지 함수
def predict_with_ood_detection(model, X, threshold=0.6):
    probs = model.predict_proba(X)  # Softmax 확률 출력
    max_probs = np.max(probs, axis=1)  # 가장 높은 확률 값
    preds = np.argmax(probs, axis=1)  # 가장 높은 확률의 클래스

    # 특정 확률(threshold) 이하이면 "기타 소음"(-1)으로 변경
    final_preds = np.where(max_probs < threshold, -1, preds)
    
    return final_preds

# 📌 6️⃣ OOD 탐지 적용 (테스트 데이터 & 기타 소음 데이터)
y_pred_test = predict_with_ood_detection(model, X_test, threshold=0.8)
y_pred_ood = predict_with_ood_detection(model, ood_X_test, threshold=0.8)

# 📌 7️⃣ 결과 출력
print("✅ 테스트 데이터 예측 결과:", y_pred_test)
print("✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링):", y_pred_ood)

✅ 테스트 데이터 예측 결과: [ 2 -1  1 ...  4  0  3]
✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링): [ 1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1]


In [11]:
df_etc.shape

(20, 60)

In [12]:
df_etc.columns

Index(['fileName', 'labelName', 'area_start', 'area_end', 'category_01',
       'category_02', 'category_03', 'decibel', 'soundQuality', 'subCategory',
       'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
       'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13',
       'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18', 'mfcc_19',
       'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23', 'mfcc_24', 'mfcc_25',
       'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29', 'mfcc_30', 'mfcc_31',
       'mfcc_32', 'mfcc_33', 'mfcc_34', 'mfcc_35', 'mfcc_36', 'mfcc_37',
       'mfcc_38', 'mfcc_39', 'mfcc_40', 'mfcc_41', 'mfcc_42', 'mfcc_43',
       'mfcc_44', 'mfcc_45', 'mfcc_46', 'mfcc_47', 'mfcc_48', 'mfcc_49',
       'mfcc_50'],
      dtype='object')

In [13]:
df_combined.shape

(16156, 82)

In [14]:
df_combined.columns

Index(['fileName', 'labelName', 'area_start', 'area_end', 'category_01',
       'category_02', 'category_03', 'decibel', 'soundQuality', 'subCategory',
       'bitRate', 'duration', 'fileFormat', 'fileSize', 'recodingType',
       'sampleRate', 'acqDevice', 'acqMethod', 'acqType', 'areaUse',
       'dayNight', 'direction', 'distance', 'district', 'latitude',
       'longitude', 'micClass', 'obstacle', 'place', 'recordingTime', 'urban',
       'weather', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6',
       'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12',
       'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18',
       'mfcc_19', 'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23', 'mfcc_24',
       'mfcc_25', 'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29', 'mfcc_30',
       'mfcc_31', 'mfcc_32', 'mfcc_33', 'mfcc_34', 'mfcc_35', 'mfcc_36',
       'mfcc_37', 'mfcc_38', 'mfcc_39', 'mfcc_40', 'mfcc_41', 'mfcc_42',
       'mfcc_43', 'mfcc_44', 'mfcc_45', 'mfcc_4

# df_etc2 기타 소음 (동물) 확인

In [15]:
import os
import json
import librosa
import numpy as np
import pandas as pd

data_dir = "/home/ubuntu/data/etc_noise_data_test_02"

# 기존 데이터프레임이 존재하는지 확인
try:
    df_combined
except NameError:
    df_combined = pd.DataFrame()  # 없으면 빈 데이터프레임 생성

json_data = []
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

            annotations = data.get("annotations", [])
            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

            # JSON 파일명에서 확장자 제거
            base_name = os.path.splitext(file)[0]

            json_data.append({
                "baseName": base_name,  # 확장자 없는 파일명 저장
                "fileName": data["audio"]["fileName"],
                "labelName": annotation.get("labelName", None),
                "area_start": annotation.get("area", {}).get("start", None),
                "area_end": annotation.get("area", {}).get("end", None),
                "category_01": annotation.get("categories", {}).get("category_01", None),
                "category_02": annotation.get("categories", {}).get("category_02", None),
                "category_03": annotation.get("categories", {}).get("category_03", None),
                "decibel": annotation.get("decibel", None),
                "soundQuality": annotation.get("soundQuality", None),
                "subCategory": annotation.get("subCategory", None),
            })

df_json = pd.DataFrame(json_data)

# WAV 파일에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=50):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

wav_data = []
for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        base_name = "_".join(file.split("_")[:-1])  # "_1.wav" 부분 제거하고 기본 파일명 추출
        mfcc_features = extract_mfcc(file_path)

        wav_data.append({
            "baseName": base_name,  # JSON과 매칭할 기본 파일명 저장
            **{f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfcc_features)}
        })

df_wav = pd.DataFrame(wav_data)

# # JSON 데이터와 WAV 데이터 병합 (baseName 기준)
df_etc2 = pd.merge(df_json, df_wav, on="baseName", how="left").drop(columns=["baseName"])

# # 기존 데이터프레임 df_combined에 추가
#df_combined2 = pd.concat([df_combined, df_etc2], ignore_index=True)

# # 결과 확인
# print(df_combined.head())

In [16]:
df_etc2.columns

Index(['fileName', 'labelName', 'area_start', 'area_end', 'category_01',
       'category_02', 'category_03', 'decibel', 'soundQuality', 'subCategory',
       'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7',
       'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13',
       'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18', 'mfcc_19',
       'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23', 'mfcc_24', 'mfcc_25',
       'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29', 'mfcc_30', 'mfcc_31',
       'mfcc_32', 'mfcc_33', 'mfcc_34', 'mfcc_35', 'mfcc_36', 'mfcc_37',
       'mfcc_38', 'mfcc_39', 'mfcc_40', 'mfcc_41', 'mfcc_42', 'mfcc_43',
       'mfcc_44', 'mfcc_45', 'mfcc_46', 'mfcc_47', 'mfcc_48', 'mfcc_49',
       'mfcc_50'],
      dtype='object')

In [17]:
file_names = df_etc2["fileName"].tolist()
print(file_names)  # 리스트 형태로 출력

['7.동물_9206.wav', '7.동물_9095.wav', '7.동물_9041.wav', '7.동물_9203.wav', '7.동물_9208.wav', '7.동물_9204.wav', '7.동물_9110.wav', '7.동물_8962.wav', '7.동물_9211.wav', '7.동물_8979.wav', '7.동물_9215.wav', '7.동물_9040.wav', '7.동물_9090.wav', '7.동물_9112.wav', '7.동물_9063.wav', '7.동물_8954.wav', '7.동물_9214.wav', '7.동물_9205.wav', '7.동물_9213.wav', '7.동물_9207.wav']


In [18]:
# 📌 4️⃣ OOD 데이터 로드 & 전처리 (MFCC 특징 사용)
ood_X_test = df_etc2.iloc[:, -50:]  # OOD 데이터의 MFCC 13개 컬럼 사용

# 📌 5️⃣ Softmax 기반 OOD 탐지 함수
def predict_with_ood_detection(model, X, threshold=0.6):
    probs = model.predict_proba(X)  # Softmax 확률 출력
    max_probs = np.max(probs, axis=1)  # 가장 높은 확률 값
    preds = np.argmax(probs, axis=1)  # 가장 높은 확률의 클래스

    # 특정 확률(threshold) 이하이면 "기타 소음"(-1)으로 변경
    final_preds = np.where(max_probs < threshold, -1, preds)
    
    return final_preds

# 📌 6️⃣ OOD 탐지 적용 (테스트 데이터 & 기타 소음 데이터)
y_pred_test = predict_with_ood_detection(model, X_test, threshold=0.8)
y_pred_ood = predict_with_ood_detection(model, ood_X_test, threshold=0.8)

# 📌 7️⃣ 결과 출력
print("✅ 테스트 데이터 예측 결과:", y_pred_test)
print("✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링):", y_pred_ood)

✅ 테스트 데이터 예측 결과: [ 2 -1  1 ...  4  0  3]
✅ OOD (기타 소음) 예측 결과 (Softmax 기반 필터링): [ 1  1  1  1  1  1  1  1  1 -1  1  1  1 -1 -1 -1  1  1  1  1]


# df_etc3 기타 소음 (df_etc1 + df_etc2) 확인

In [9]:
import os
import json
import librosa
import numpy as np
import pandas as pd

data_dir = "/home/ubuntu/data/etc_noise_data_test_03"

# 기존 데이터프레임이 존재하는지 확인
try:
    df_combined
except NameError:
    df_combined = pd.DataFrame()  # 없으면 빈 데이터프레임 생성

json_data = []
for file in os.listdir(data_dir):
    if file.endswith(".json"):
        with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
            data = json.load(f)

            annotations = data.get("annotations", [])
            annotation = annotations[0] if annotations and isinstance(annotations, list) else {}

            # JSON 파일명에서 확장자 제거
            base_name = os.path.splitext(file)[0]

            json_data.append({
                "baseName": base_name,  # 확장자 없는 파일명 저장
                "fileName": data["audio"]["fileName"],
                "labelName": annotation.get("labelName", None),
                "area_start": annotation.get("area", {}).get("start", None),
                "area_end": annotation.get("area", {}).get("end", None),
                "category_01": annotation.get("categories", {}).get("category_01", None),
                "category_02": annotation.get("categories", {}).get("category_02", None),
                "category_03": annotation.get("categories", {}).get("category_03", None),
                "decibel": annotation.get("decibel", None),
                "soundQuality": annotation.get("soundQuality", None),
                "subCategory": annotation.get("subCategory", None),
            })

df_json = pd.DataFrame(json_data)

# WAV 파일에서 MFCC 추출하는 함수
def extract_mfcc(file_path, sr=22050, n_mfcc=50):
    try:
        y, sr = librosa.load(file_path, sr=sr)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs, axis=1)  # MFCC 평균값 반환
    except Exception as e:
        print(f"오류 발생: {file_path} - {e}")
        return [None] * n_mfcc  # 오류 발생 시 None 리스트 반환

wav_data = []
for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        base_name = "_".join(file.split("_")[:-1])  # "_1.wav" 부분 제거하고 기본 파일명 추출
        mfcc_features = extract_mfcc(file_path)

        wav_data.append({
            "baseName": base_name,  # JSON과 매칭할 기본 파일명 저장
            **{f"mfcc_{i+1}": mfcc for i, mfcc in enumerate(mfcc_features)}
        })

df_wav = pd.DataFrame(wav_data)

# # JSON 데이터와 WAV 데이터 병합 (baseName 기준)
df_etc3 = pd.merge(df_json, df_wav, on="baseName", how="left").drop(columns=["baseName"])

# # 기존 데이터프레임 df_combined에 추가
#df_combined2 = pd.concat([df_combined, df_etc2], ignore_index=True)

# # 결과 확인
# print(df_combined.head())

In [10]:
df_etc3.shape

(40, 60)

In [11]:
file_names = df_etc3["fileName"].tolist()
print(file_names)  # 리스트 형태로 출력

['7.동물_9206.wav', '7.동물_9095.wav', '7.동물_9041.wav', '3.항공기_3089.wav', '3.항공기_3096.wav', '7.동물_9203.wav', '4.열차_3548.wav', '7.동물_9208.wav', '7.동물_9204.wav', '7.동물_9110.wav', '3.항공기_65672.wav', '7.동물_8962.wav', '3.항공기_19365.wav', '7.동물_9211.wav', '3.항공기_3090.wav', '4.열차_3551.wav', '7.동물_8979.wav', '3.항공기_65674.wav', '4.열차_3807.wav', '7.동물_9215.wav', '4.열차_3806.wav', '7.동물_9040.wav', '4.열차_3554.wav', '3.항공기_65671.wav', '7.동물_9090.wav', '7.동물_9112.wav', '3.항공기_3093.wav', '3.항공기_65673.wav', '7.동물_9063.wav', '4.열차_3810.wav', '4.열차_3552.wav', '4.열차_3808.wav', '7.동물_8954.wav', '3.항공기_3092.wav', '7.동물_9214.wav', '7.동물_9205.wav', '4.열차_3809.wav', '7.동물_9213.wav', '7.동물_9207.wav', '4.열차_3547.wav']


In [12]:
# 📌 4️⃣ OOD 데이터 로드 & 전처리 (MFCC 특징 사용)
ood_X_test = df_etc3.iloc[:, -50:]  # OOD 데이터의 MFCC 13개 컬럼 사용

# 📌 5️⃣ Softmax 기반 OOD 탐지 함수
def predict_with_ood_detection(model, X, threshold=0.6):
    probs = model.predict_proba(X)  # Softmax 확률 출력
    max_probs = np.max(probs, axis=1)  # 가장 높은 확률 값
    preds = np.argmax(probs, axis=1)  # 가장 높은 확률의 클래스

    # 특정 확률(threshold) 이하이면 "기타 소음"(-1)으로 변경
    final_preds = np.where(max_probs < threshold, -1, preds)
    
    return final_preds

# 📌 6️⃣ OOD 탐지 적용 (기타 소음 데이터)
y_pred_ood = predict_with_ood_detection(model, ood_X_test, threshold=0.8)

# ✅ 예측 결과를 데이터프레임에 추가하여 파일명과 함께 출력
df_etc3["predicted_label"] = y_pred_ood

# ✅ 기타 소음(-1) 개수 카운트
num_neg1_ood = np.count_nonzero(y_pred_ood == -1)

# ✅ 출력 정리 (파일명 + 예측 결과)
print("\n📌 OOD (기타 소음) 예측 결과 (Softmax 기반 필터링)")
print(df_etc3[["fileName", "predicted_label"]].to_string(index=False))

# ✅ 기타 소음(-1) 개수 출력
print(f"\n✅ OOD 데이터에서 -1 개수: {num_neg1_ood}")


📌 OOD (기타 소음) 예측 결과 (Softmax 기반 필터링)
        fileName  predicted_label
   7.동물_9206.wav                1
   7.동물_9095.wav                1
   7.동물_9041.wav                1
  3.항공기_3089.wav                1
  3.항공기_3096.wav                1
   7.동물_9203.wav                1
   4.열차_3548.wav                1
   7.동물_9208.wav                1
   7.동물_9204.wav                1
   7.동물_9110.wav                1
 3.항공기_65672.wav                1
   7.동물_8962.wav                1
 3.항공기_19365.wav                1
   7.동물_9211.wav                1
  3.항공기_3090.wav                1
   4.열차_3551.wav                1
   7.동물_8979.wav               -1
 3.항공기_65674.wav               -1
   4.열차_3807.wav                1
   7.동물_9215.wav                1
   4.열차_3806.wav                1
   7.동물_9040.wav                1
   4.열차_3554.wav                1
 3.항공기_65671.wav                1
   7.동물_9090.wav                1
   7.동물_9112.wav               -1
  3.항공기_3093.wav                1
 3.항공기_656

# Autoencoder를 이용한 OOD 탐지 및 차원 축소 후 LightGBM을 활용한 분류 모델 학습

In [24]:
print(df_combined['category_03'].unique())
print(df_etc3['category_03'].unique())

['차량경적' '차량사이렌' '차량주행음' '이륜차경적' '이륜차주행음']
['고양이' '개' '비행기' '기차' '헬리콥터' '지하철']


In [25]:
print(f"ood_X_test 샘플 개수: {len(ood_X_test)}")
print(f"df_etc3 샘플 개수: {len(df_etc3)}")

ood_X_test 샘플 개수: 40
df_etc3 샘플 개수: 40


In [26]:
print(ood_X_test.shape)  # 데이터 크기 확인
print(ood_X_test.head())  # 데이터 일부 샘플 확인

(40, 50)
       mfcc_1      mfcc_2     mfcc_3    mfcc_4     mfcc_5     mfcc_6  \
0 -294.744812  125.595810 -28.943182  9.932528   0.388238   8.065389   
1 -320.291321  161.263351 -28.671167 -1.095764  -4.097131  11.460157   
2 -338.099274  118.241966 -21.588984  4.763221  -3.009731  11.009373   
3 -493.554016  209.486206  41.519714  1.461567  19.411510  11.984865   
4 -326.547974  238.013596  37.362270 -6.802079   7.256361  19.280979   

      mfcc_7     mfcc_8     mfcc_9    mfcc_10  ...   mfcc_41   mfcc_42  \
0 -10.260452  -1.014173 -10.899606   2.571267  ... -2.791410 -0.157648   
1  -0.975062  12.972774  -5.271904   3.701254  ... -2.182304  0.485709   
2   6.188801  -2.684155  -0.431702  11.627713  ... -0.197318 -1.002083   
3  -3.360848   7.662577  -8.510960   2.700191  ... -1.559616 -2.330000   
4  -3.648936  -8.836903   3.380370   4.751110  ... -1.255778 -1.950921   

    mfcc_43   mfcc_44   mfcc_45   mfcc_46   mfcc_47   mfcc_48   mfcc_49  \
0 -0.891546 -0.091464 -1.380870  0.214

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

ood_X_test = df_etc3.iloc[:, -50:]

# 📌 1️⃣ Autoencoder 모델 정의
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=16):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# ✔️ Autoencoder(오토인코더)는 입력 데이터를 압축(encoding)했다가 다시 복원(decoding)하는 신경망 모델입니다.
# ✔️ encoder는 입력 데이터를 16차원(latent_dim=16)으로 축소합니다.
# ✔️ decoder는 다시 원래 차원(input_dim=50)으로 복원합니다.    
    
    
# 📌 2️⃣ 데이터 준비
X = df_combined.iloc[:, -50:].values  # MFCC 50개 특성 사용
y = df_combined['category_03'].astype('category').cat.codes.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✔️ df_combined 데이터프레임에서 마지막 50개 열을 특징(X)으로 사용합니다.
# ✔️ category_03을 정수형 레이블로 변환하여 y로 저장합니다.
# ✔️ 80%의 데이터를 X_train, y_train, 나머지 20%를 X_test, y_test로 나눕니다.


# ✅ 강제로 CPU 사용
device = torch.device("cpu")
# PyTorch Tensor 변환
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

#✔️ 데이터를 PyTorch Tensor 형태로 변환하여 Autoencoder에 입력할 준비를 합니다.


# 📌 3️⃣ Autoencoder 학습
input_dim = X_train.shape[1]
latent_dim = 16  # 압축할 차원 설정
model = Autoencoder(input_dim, latent_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, X_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# ✔️ Autoencoder를 학습시키기 위해 손실 함수(MSE)와 Adam 최적화 함수를 사용합니다.
# ✔️ 50번의 학습(epoch)을 반복하면서 모델을 최적화합니다.
# ✔️ 10 epoch마다 현재 재구성 오차(loss)를 출력합니다.
        
        
        
# 📌 4️⃣ OOD 탐지 - 재구성 오차 계산
model.eval()
def detect_ood(X, threshold=0.05):
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    with torch.no_grad():
        reconstructed = model(X_tensor)
    loss = torch.mean((X_tensor - reconstructed) ** 2, axis=1).numpy()
    return np.where(loss > threshold, -1, 1)  # 임계값보다 크면 OOD

# ✔️ Autoencoder를 활용하여 OOD 데이터를 탐지하는 방법:
# 입력 데이터를 Autoencoder에 통과시켜 재구성합니다.
# 원본 데이터와 복원된 데이터 간의 재구성 오차(MSE)를 계산합니다.
# 오차가 임계값(threshold)보다 크면 OOD(-1), 작으면 정상(1)으로 간주합니다.


# OOD 탐지 적용
ood_labels = detect_ood(ood_X_test.values, threshold=0.5)
print("OOD 탐지 결과:", ood_labels)


# OOD 탐지 결과에서 -1의 개수와 비율 계산
num_ood = np.sum(ood_labels == -1)  # -1인 사바 개수
total_samples = len(ood_labels)  # 전체 사바 개수
ood_ratio = num_ood / total_samples  # OOD 비율

# 결과 출력
print(f"OOD 탐지 결과 -1 개수: {num_ood}")
print(f"OOD 비율: {ood_ratio:.4%}")  #ood_X_test에 있는 데이터가 기존 학습 데이터와 차이가 크다는 의미.

# 📌 5️⃣ 차원 축소된 데이터로 LightGBM 학습
# with torch.no_grad():
#     X_train_reduced = model.encoder(X_train_tensor).cpu().numpy()
#     X_test_reduced = model.encoder(X_test_tensor).cpu().numpy()


with torch.no_grad():
    X_train_reduced = model.encoder(X_train_tensor.to(device)).numpy()
    X_test_reduced = model.encoder(X_test_tensor.to(device)).numpy()

#✔️ Autoencoder의 Encoder 부분만 사용하여 데이터를 16차원(latent_dim=16)으로 축소.

lgbm = LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
lgbm.fit(X_train_reduced, y_train)
y_pred = lgbm.predict(X_test_reduced)

# ✔️ LightGBM 모델을 사용하여 분류 학습을 진행합니다.
# ✔️ 차원 축소된 데이터를 활용해 학습한 후 y_pred를 예측합니다.


# 결과 출력
acc = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도 (Autoencoder 차원 축소 적용): {acc:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Epoch [10/50], Loss: 2574.6399
Epoch [20/50], Loss: 2538.9365
Epoch [30/50], Loss: 2452.7192
Epoch [40/50], Loss: 2283.9087
Epoch [50/50], Loss: 1960.4109
OOD 탐지 결과: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
OOD 탐지 결과 -1 개수: 40
OOD 비율: 100.0000%
LightGBM 정확도 (Autoencoder 차원 축소 적용): 0.8475
Confusion Matrix:
[[851  22  69   6   1]
 [  3 807  15  24  66]
 [ 68  16 507  14  16]
 [  3  34  12 360  13]
 [  2  97   6   6 214]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       949
           1       0.83      0.88      0.85       915
           2       0.83      0.82      0.82       621
           3       0.88      0.85      0.87       422
           4       0.69      0.66      0.67       325

    accuracy                           0.85      3232
   macro avg       0.83      0.82      0.82      3232
weighted avg       0.85      0.85     

📌 결론
1. Autoencoder를 활용한 차원 축소 성공 → LightGBM 적용하여 85.95% 정확도 달성.
2. OOD 탐지에서 100% 이상 탐지 → OOD 데이터가 기존 학습 데이터와 상당한 차이가 있음.
3. Confusion Matrix에서 일부 클래스 성능이 낮음 → 데이터 불균형을 확인할 필요 있음.

✔ 다음 단계:
- OOD 탐지 임계값(threshold)을 조정하여 탐지 정확도 개선.
- 클래스 간 불균형 문제 해결 (데이터 증강, 가중치 조정 등).

In [28]:
❓ 100% OOD 탐지된 데이터가 df_etc3인지, df_combined인지?
OOD 탐지에 사용된 데이터는 df_etc3의 마지막 50개 특성 (ood_X_test) 입니다.

💡 즉, 탐지된 -1 결과는 df_etc3에 있는 데이터에서 나온 것입니다.

df_etc3가 기타 소음이라면 모두 OOD로 탐지되는 것이 정상적인 결과일 가능성이 큽니다.
df_combined 데이터로 학습한 Autoencoder가 df_etc3를 기존 분포에서 벗어난 데이터로 인식한 것입니다.
⚠️ 하지만 100% OOD가 나오는 게 예상보다 과도하다면?
df_etc3가 정말 학습에 사용된 df_combined의 분포와 다른 데이터인지 확인해야 합니다.
데이터가 같은 종류라면 Autoencoder가 과적합되었거나 임계값이 너무 낮을 가능성도 있습니다.
🔹 확인할 방법:
1️⃣ df_etc3와 df_combined의 데이터를 시각화해서 분포 비교
2️⃣ OOD 임계값(현재 0.8)을 조정해서 OOD 결과 변화를 확인
3️⃣ Autoencoder가 정상 데이터를 얼마나 잘 복원하는지 확인 (df_combined 테스트 데이터에 대한 재구성 오차 분석)

🚀 결론
100% OOD로 탐지된 데이터는 df_etc3에서 가져온 ood_X_test입니다.
만약 df_etc3가 실제로 기타 소음이라면, 이 결과는 정상적입니다.
그러나 같은 유형의 데이터라면 Autoencoder의 일반화 문제일 가능성이 있어 추가 확인이 필요합니다.

SyntaxError: invalid syntax (<ipython-input-28-600a9416aa99>, line 2)

In [29]:
# OOD로 탐지된 데이터 인덱스 찾기
ood_indices = np.where(ood_labels == -1)[0]  # -1인 데이터의 인덱스

# OOD로 분류된 데이터 추출 (파일명 + 카테고리 확인)
ood_data_with_filenames = df_combined.iloc[ood_X_test.iloc[ood_indices].index][['fileName', 'category_03']]

# 결과 출력
print(ood_data_with_filenames)


                  fileName category_03
0   1.car_horn_87900_1.wav        차량경적
1   1.car_horn_87964_1.wav        차량경적
2   1.car_horn_87987_1.wav        차량경적
3   1.car_horn_88000_1.wav        차량경적
4   1.car_horn_88001_1.wav        차량경적
5   1.car_horn_87669_1.wav        차량경적
6   1.car_horn_87808_1.wav        차량경적
7   1.car_horn_87809_1.wav        차량경적
8   1.car_horn_87814_1.wav        차량경적
9   1.car_horn_87818_1.wav        차량경적
10  1.car_horn_87819_1.wav        차량경적
11  1.car_horn_87820_1.wav        차량경적
12  1.car_horn_87821_1.wav        차량경적
13  1.car_horn_87824_1.wav        차량경적
14  1.car_horn_87825_1.wav        차량경적
15  1.car_horn_87828_1.wav        차량경적
16  1.car_horn_87833_1.wav        차량경적
17  1.car_horn_87834_1.wav        차량경적
18  1.car_horn_87835_1.wav        차량경적
19  1.car_horn_87836_1.wav        차량경적
20  1.car_horn_87839_1.wav        차량경적
21  1.car_horn_87840_1.wav        차량경적
22  1.car_horn_87841_1.wav        차량경적
23  1.car_horn_87842_1.wav        차량경적
24  1.car_horn_87845_1.wa

In [ ]:
🚨 정상 데이터(차량경적)가 OOD로 탐지된 문제 분석
현재 차량경적 소음 데이터가 OOD(-1)로 탐지되었는데, 이는 예상과 다른 결과입니다.
즉, df_combined에서 학습한 Autoencoder가 정상 데이터마저 OOD로 분류하는 현상이 발생했습니다.

In [ ]:
print(" *** 테스트 데이터 *** ")
etc = 0
non_etc = 0 
total = 0 
for i in y_pred_test :
    if i == -1 :
        etc += 1
    else :
        non_etc += 1 
    total += 1
    
    etc_ratio = etc / total * 100  
    non_etc_ratio = non_etc / total * 100  

print(f"기타 소음 개수: {etc}")
print(f"기타 소음 처리가 안된 소음 개수: {non_etc}")
print(f"전체 소음 개수: {total}")

print(f"기타 소음 비율: {etc_ratio:.2f}%")
print(f"비기타 소음 비율: {non_etc_ratio:.2f}%")

print( "*" * 30 )
print(" *** 기타 소음 데이터 *** ")
etc = 0
non_etc = 0 
total = 0 
for i in y_pred_ood :
    if i == -1 :
        etc += 1
    else :
        non_etc += 1 
    total += 1
    
    etc_ratio = etc / total * 100  
    non_etc_ratio = non_etc / total * 100  

print(f"기타 소음 처리된 개수: {etc}")
print(f"기타 소음 처리가 안된 소음 개수: {non_etc}")
print(f"전체 소음 개수: {total}")

print(f"기타 소음 비율: {etc_ratio:.2f}%")
print(f"비기타 소음 비율: {non_etc_ratio:.2f}%")

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 📌 기존 데이터 사용
X = df_combined.iloc[:, -50:]  # MFCC 특징 사용
y = df_combined['category_03'].astype('category').cat.codes

# 📌 데이터 정규화 (Autoencoder 학습을 위해 0-1 스케일링)
X = (X - X.min()) / (X.max() - X.min())

# 📌 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Autoencoder 모델 정의
input_dim = X.shape[1]
latent_dim = 10  # 저차원 표현 차원

encoder = keras.Sequential([
    keras.layers.Input(shape=(input_dim,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(latent_dim, activation='relu')  # 잠재 공간 표현
])

decoder = keras.Sequential([
    keras.layers.Input(shape=(latent_dim,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(input_dim, activation='sigmoid')  # 재구성
])

autoencoder = keras.Model(inputs=encoder.input, outputs=decoder(encoder.output))
autoencoder.compile(optimizer='adam', loss='mse')

# ✅ Autoencoder 학습
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test), verbose=1)

# ✅ 인코딩된 차원 축소 특징 추출
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)


Epoch 1/50
404/404 [==============================] - 2s 2ms/step - loss: 0.0115 - val_loss: 0.0097
Epoch 2/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0085 - val_loss: 0.0082
Epoch 3/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0071 - val_loss: 0.0069
Epoch 4/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0064 - val_loss: 0.0064
Epoch 5/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0060 - val_loss: 0.0061
Epoch 6/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0058 - val_loss: 0.0059
Epoch 7/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 8/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0054 - val_loss: 0.0055
Epoch 9/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0053 - val_loss: 0.0054
Epoch 10/50
404/404 [==============================] - 1s 2ms/step - loss: 0.0052 - val_loss: 0.0052

In [14]:
# ✅ LightGBM 모델 학습
model = LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_encoded, y_train)

# ✅ 예측 및 평가
y_pred = model.predict(X_test_encoded)

# ✅ 평가 결과 출력
acc = accuracy_score(y_test, y_pred)
print(f"LightGBM 정확도: {acc:.4f}")

# ✅ 혼동 행렬 및 분류 보고서 출력
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


LightGBM 정확도: 0.8666

Confusion Matrix:
[[874  18  53   1   3]
 [  2 808  16  11  78]
 [ 32  45 530   6   8]
 [  1  18   8 380  15]
 [  0 107   4   5 209]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       949
           1       0.81      0.88      0.85       915
           2       0.87      0.85      0.86       621
           3       0.94      0.90      0.92       422
           4       0.67      0.64      0.66       325

    accuracy                           0.87      3232
   macro avg       0.85      0.84      0.84      3232
weighted avg       0.87      0.87      0.87      3232



In [17]:
print(f"ood_X_test.shape: {ood_X_test.shape}")


ood_X_test.shape: (40, 51)


In [18]:
print(df_etc3.columns[-50:])  # 마지막 50개 컬럼 이름 확인


Index(['mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8',
       'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'mfcc_14',
       'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18', 'mfcc_19', 'mfcc_20',
       'mfcc_21', 'mfcc_22', 'mfcc_23', 'mfcc_24', 'mfcc_25', 'mfcc_26',
       'mfcc_27', 'mfcc_28', 'mfcc_29', 'mfcc_30', 'mfcc_31', 'mfcc_32',
       'mfcc_33', 'mfcc_34', 'mfcc_35', 'mfcc_36', 'mfcc_37', 'mfcc_38',
       'mfcc_39', 'mfcc_40', 'mfcc_41', 'mfcc_42', 'mfcc_43', 'mfcc_44',
       'mfcc_45', 'mfcc_46', 'mfcc_47', 'mfcc_48', 'mfcc_49', 'mfcc_50',
       'predicted_label'],
      dtype='object')


In [19]:
# ✅ OOD 데이터 (기타 소음) 처리
ood_X_test = df_etc3.iloc[:, -50:-1]  # OOD 데이터 MFCC 특징 사용
ood_X_test = (ood_X_test - X.min()) / (X.max() - X.min())  # 정규화

# ✅ Autoencoder 재구성 오차 계산
reconstructed = autoencoder.predict(ood_X_test)
reconstruction_error = np.mean(np.abs(reconstructed - ood_X_test), axis=1)

# ✅ OOD 탐지 (재구성 오차가 특정 임계값 이상이면 OOD로 판정)
threshold = np.percentile(reconstruction_error, 95)  # 상위 5%를 OOD로 설정
ood_preds = np.where(reconstruction_error > threshold, -1, model.predict(encoder.predict(ood_X_test)))

# ✅ 결과 저장 및 분석
df_etc3["predicted_label"] = ood_preds
num_ood = np.count_nonzero(ood_preds == -1)

print(f"\n✅ OOD 데이터에서 탐지된 개수: {num_ood}")
print(df_etc3[["fileName", "predicted_label"]].to_string(index=False))



✅ OOD 데이터에서 탐지된 개수: 0
        fileName  predicted_label
   7.동물_9206.wav                2
   7.동물_9095.wav                2
   7.동물_9041.wav                2
  3.항공기_3089.wav                2
  3.항공기_3096.wav                2
   7.동물_9203.wav                2
   4.열차_3548.wav                2
   7.동물_9208.wav                2
   7.동물_9204.wav                2
   7.동물_9110.wav                2
 3.항공기_65672.wav                2
   7.동물_8962.wav                2
 3.항공기_19365.wav                2
   7.동물_9211.wav                2
  3.항공기_3090.wav                2
   4.열차_3551.wav                2
   7.동물_8979.wav                2
 3.항공기_65674.wav                2
   4.열차_3807.wav                2
   7.동물_9215.wav                2
   4.열차_3806.wav                2
   7.동물_9040.wav                2
   4.열차_3554.wav                2
 3.항공기_65671.wav                2
   7.동물_9090.wav                2
   7.동물_9112.wav                2
  3.항공기_3093.wav                2
 3.항공기_65673.wav         